In [1]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
import numpy as np
import pandas as pd
import networkx as nx
from rdkit.Chem.rdchem import BondType


In [2]:
df_tox = pd.read_csv('toxic_small_0105.csv')
df_exo = pd.read_csv('exo_small_0105.csv')

In [3]:
#将外源性分子中有毒性分子的排除
tox_smi_list = set(df_tox['standard_smiles'].values)
exo_smi_list = set(df_exo['standard_smiles'].values)
print(len(tox_smi_list), len(exo_smi_list))
exo_smi_list = exo_smi_list - tox_smi_list
tox_smi_list = list(tox_smi_list)
exo_smi_list = list(exo_smi_list)
print(len(tox_smi_list), len(exo_smi_list))

3404 7519
3404 7235


In [4]:
tox_mols = [Chem.MolFromSmiles(smi) for smi in tox_smi_list]
exo_mols = [Chem.MolFromSmiles(smi) for smi in exo_smi_list]

[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors
[02:34:01] WARNING: not removing hydrogen atom without neighbors


In [11]:
def GetNBEandGapEnergy(mols, smiles_list, res_error='tox_gapenergy_error.txt',):
    res_error = open(res_error,'a')
    descs = [desc_name[0] for desc_name in Descriptors._descList]
    desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(descs)
    atoms = [['H', 'H'], ['H', 'F'], ['H', 'Cl'], ['H', 'Br'], ['H', 'I'],
             ['C', 'H'], ['C', 'C'], ['C', 'N'], ['C', 'O'], ['C', 'F'], ['C', 'Cl'], ['C', 'Br'], ['C', 'I'], ['C', 'S'],
             ['N', 'H'], ['N', 'N'], ['N', 'F'], ['N', 'Cl'], ['N', 'Br'], ['N', 'O'],
             ['O', 'H'], ['O', 'O'], ['O', 'F'], ['O', 'Cl'], ['O', 'I'],
             ['F', 'F'], ['F', 'Cl'], ['F', 'Br'], ['Cl', 'Cl'], ['Cl', 'Br'], ['Br', 'Br'], ['I', 'I'], ['I', 'Cl'], ['I', 'Br'],
             ['S', 'H'], ['S', 'F'], ['S', 'Cl'], ['S', 'Br'], ['S', 'S'],
             ['Si', 'Si'], ['Si', 'H'], ['Si', 'C'], ['Si', 'O'],
             ['C', 'C'], ['C', 'C'], ['O', 'O'], ['C', 'O'], ['C', 'O'], ['N', 'O'], ['N', 'N'],
             ['N', 'N'], ['C', 'N'], ['C', 'N'],
             ['P', 'H'], ['P', 'Cl'], ['P', 'Br'], ['P', 'O'], ['P', 'O'], ['P', 'P'], ['P', 'C'],
             ['S', 'C'], ['S', 'O'], ['S', 'O'],
             ['S','N'], ['S','P'], ['S','P']]
    bondtypes = [BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, # 5
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, #9
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, #6
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, #5
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, #9
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, #5
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, #4
                 BondType.DOUBLE, BondType.TRIPLE, BondType.DOUBLE, BondType.DOUBLE, BondType.TRIPLE, BondType.DOUBLE, BondType.DOUBLE, #7
                 BondType.TRIPLE, BondType.TRIPLE, BondType.DOUBLE, #3
                 BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.SINGLE, BondType.DOUBLE, BondType.SINGLE, BondType.SINGLE, #7
                 BondType.DOUBLE, BondType.SINGLE, BondType.DOUBLE, #3
                 BondType.SINGLE, BondType.SINGLE, BondType.DOUBLE #3
                 ]
    energies = [432, 565, 427, 363, 295, 413, 347, 305, 358, 485, 339, 276, 240, 259, 391, 160, 272, 200, 243, 201, 467,
                146, 190, 203, 234,
                154, 253, 237, 239, 218, 193, 149, 208, 175, 347, 327, 253, 218, 266, 340, 393, 360, 452, 614, 839, 495,
                745, 1072, 607, 418,
                941, 891, 615, 322, 331, 272, 410, 585, 213, 305, 536, 226, 460,
                121, 463, 389]
    # 键长参考https://wenku.baidu.com/view/d9284346767f5acfa1c7cd42.html#opennewwindow
    # http://www.wiredchemist.com/chemistry/data/bond_energies_lengths.html
    # https://cccbdb.nist.gov/calcbondcomp2.asp?i=7&j=35
    ExactbondWet = [2.015650064, 20.006228252, 35.976677712, 79.926162132, 127.912298032, 13.007825032, 24.0,
                    26.003073999999998, 27.99491462, 30.99840322, 46.96885268,
                    90.9183371, 138.904473, 43.972071, 15.010899032, 28.006148, 33.00147722, 48.971926679999996,
                    92.9214111, 29.99798862, 17.002739652, 31.98982924,
                    34.99331784, 50.9637673, 142.89938762, 37.99680644, 53.9672559, 97.91674032, 69.93770536,
                    113.88718978, 157.8366742, 253.808946,
                    161.87332568, 205.8228101, 32.979896032, 50.97047422, 66.94092368, 110.8904081, 63.944142,
                    55.95385306, 28.984751562, 39.97692653,
                    43.97184115, 24.0, 24.0, 31.98982924, 27.99491462, 27.99491462, 29.99798862, 28.006148, 28.006148,
                    26.003073999999998, 26.003073999999998,
                    31.981586661999998, 65.94261431, 109.89209873, 46.96867625, 46.96867625, 61.94752326, 42.97376163, \
                    43.972071, 47.96698562, 47.96698562,
                    45.975145, 62.94583263, 62.94583263]

    icount = 0
    suscount = 0
    errcount = 0
    nbegapenergy = []
    while True:

        if icount>= len(smiles_list):
            break
        mol = mols[icount]

        try:
            bindnum = len(mol.GetBonds())
            if bindnum > 2:
                pass
            else:
                res_error.write(str(icount) + '\t' + 'num_bond < 3.' + '\t' + str(smiles_list[icount]) + '\n')
                errcount += 1
                del mol[icount]
                del smiles_list[icount]

                continue
        except:
            res_error.write(str(icount) + '\t' + 'cannot get bond.' + '\t' + str(smiles_list[icount]) + '\n')
            errcount += 1
            del mols[icount]
            del smiles_list[icount]
            continue

        try:
            mol1 = Chem.AddHs(mol)  # RDkit中默认不显示氢,向分子中添加H
        except:
            res_error.write(str(icount) + '\t' + 'AddHs error.' + '\t' + str(smiles_list[icount]) + '\n')
            errcount += 1
            del mols[icount]
            del smiles_list[icount]
            continue
        try:
            Chem.Kekulize(mol1)  # 向分子中添加芳香共轭键
        except:
            res_error.write(str(icount) + '\t' + 'Addkekus error.' + '\t' + str(smiles_list[icount]) + '\n')
            errcount += 1
            del mols[icount]
            del smiles_list[icount]
            continue

        flag = False
        newbondenergy = []
        oldbondenergy = []
        list_of_numatompair = []
        for bond in mol1.GetBonds():  # 读取mol中所有化学键，得到所有键的列表，遍历
            flag = False
            bondenergy = 0.0
            beginnum = mol1.GetAtomWithIdx(bond.GetBeginAtomIdx()).GetIdx()
            endnum = mol1.GetAtomWithIdx(bond.GetEndAtomIdx()).GetIdx()
            atompair = [mol1.GetAtomWithIdx(bond.GetBeginAtomIdx()).GetSymbol(),mol1.GetAtomWithIdx(bond.GetEndAtomIdx()).GetSymbol()]

            for i in range(len(atoms)):  # 一个个试，atoms是数据库里的已知所有的原子对
                if (sorted(atompair) == sorted(atoms[i])) and (bond.GetBondType() == bondtypes[i]):  # 一个个比对
                    flag = True
                    bondenergy = energies[i]
                    newbondenergy.append(bondenergy / ExactbondWet[i])
                    oldbondenergy.append(bondenergy)
            if flag == True:
                list_of_numatompair.append([beginnum, endnum, bondenergy])
                pass
            else:
                res_error.write(
                    str(icount) + '\t' + f'cannot calculate bond energy {atompair[0]} and {atompair[1]},{bond.GetBondType()}.' + '\t'  +
                    str(smiles_list[icount]) + '\n')
                errcount += 1
                break
        if flag == False:
            del mols[icount]
            del smiles_list[icount]
            continue
        else:
            pass

        newstd = np.std(newbondenergy)
        newnbe = sum(newbondenergy) / len(mol1.GetBonds())
        oldstd = np.std(oldbondenergy)
        oldnbe = sum(oldbondenergy) / Descriptors.ExactMolWt(mol1)

        G = nx.Graph()
        num = len(list_of_numatompair)

        nodelist = [[i, list_of_numatompair[i][2]] for i in range(num)]

        edgelist = []
        for i in range(num):
            a1 = list_of_numatompair[i][0]
            b1 = list_of_numatompair[i][1]
            for x in range(num):
                if list_of_numatompair[x][0] == a1 or list_of_numatompair[x][1] == a1:
                    edgelist.append((i, x))
                if list_of_numatompair[x][0] == b1 or list_of_numatompair[x][1] == b1:
                    edgelist.append((i, x))
        edgelistdel = []
        for edge in edgelist:
            if edge[0] == edge[1]:
                edgelistdel.append(edge)
        for edgedel in edgelistdel:
            edgelist.remove(edgedel)


        G.add_nodes_from([i[0] for i in nodelist])
        G.add_edges_from(edgelist)

        pathlist = []
        flag = True

        length_source_target = dict(nx.shortest_path_length(G))
        gap_and_energy = []
        for idx1 in range(num):
            for idx2 in range(idx1+1,num):
                try:
                    gap = length_source_target[idx1][idx2]
                    startenergy = nodelist[idx1][1]
                    endenergy = nodelist[idx2][1]
                    gapenergy = abs(startenergy-endenergy)
                    gap_and_energy.append([gap, gapenergy])
                except:
                    res_error.write(str(icount) + '\t' + smiles_list[icount]+ '\t' +f'exist node without path {idx1} and {idx2}'+'\n')
                    flag = False
                    break
            if flag == False:
                break

        if flag == False:
            errcount += 1
            del mols[icount]
            del smiles_list[icount]
            continue

    
        max_num = 1 # max_num是一个分子的最大gap
        for i in gap_and_energy:
            max_num = max(max_num, i[0])
        # print(max_num)
        # arrlist是储存各个gap下平均键能差的数组的列表
        arrdata = []
        for i in range(max_num):  # range(max_num)
            arrlist = []
            for x in gap_and_energy:
                if x[0] == i + 1:
                    arrlist.append(x[1])
            arrdata.append(arrlist)

        dimension = 10

        nbegapenergy_mol=[]
        #sxmean, sxstd, nxmean, nxstd
        for i in range(dimension):
            try:
                nbegapenergy_mol.append(np.mean(arrdata[i]))
            except:
                nbegapenergy_mol.append(np.nan)
        for i in range(dimension):
            try:
                nbegapenergy_mol.append(np.std(arrdata[i]))
            except:
                nbegapenergy_mol.append(np.nan) 
        for i in range(dimension):
            try:
                nbegapenergy_mol.append(np.mean(arrdata[-i-1]))
            except:
                nbegapenergy_mol.append(np.nan) 
        for i in range(dimension):
            try:
                nbegapenergy_mol.append(np.std(arrdata[-i-1]))
            except:
                nbegapenergy_mol.append(np.nan) 
        nbegapenergy_mol.extend([oldnbe, oldstd, newnbe, newstd, max_num])
        nbegapenergy.append(nbegapenergy_mol)
        icount += 1
        suscount += 1
    return nbegapenergy, smiles_list


In [12]:
tox_nbegapenergy1, tox_smiles_list1 =GetNBEandGapEnergy(tox_mols, tox_smi_list)
exo_nbegapenergy1, exo_smiles_list1 =GetNBEandGapEnergy(exo_mols, exo_smi_list)

In [14]:
col_list = ['sx_ave1', 'sx_ave2','sx_ave3','sx_ave4','sx_ave5',
            'sx_ave6',  'sx_ave7','sx_ave8','sx_ave9','sx_ave10',
            'sx_std1', 'sx_std2', 'sx_std3', 'sx_std4', 'sx_std5', 
            'sx_std6', 'sx_std7', 'sx_std8', 'sx_std9', 'sx_std10', 
            'nx_ave1', 'nx_ave2','nx_ave3','nx_ave4','nx_ave5',
            'nx_ave6',  'nx_ave7','nx_ave8','nx_ave9','nx_ave10',
            'nx_std1', 'nx_std2', 'nx_std3', 'nx_std4', 'nx_std5', 
            'nx_std6', 'nx_std7', 'nx_std8', 'nx_std9', 'nx_std10', 
            'oldnbe', 'oldstd', 'newnbe', 'newstd', 'max_gap']
df_tox_smi = pd.DataFrame({'standard_smiles':tox_smiles_list1})
df_tox_gapenergy = pd.DataFrame(tox_nbegapenergy1, columns= col_list)
df_tox = pd.concat([df_tox_smi, df_tox_gapenergy],axis=1)

df_exo_smi = pd.DataFrame({'standard_smiles':exo_smiles_list1})
df_exo_gapenergy = pd.DataFrame(exo_nbegapenergy1, columns= col_list)
df_exo = pd.concat([df_exo_smi, df_exo_gapenergy],axis=1)
df_exo.head()

,standard_smiles,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,...,nx_std6,nx_std7,nx_std8,nx_std9,nx_std10,oldnbe,oldstd,newnbe,newstd,max_gap
0,CC1CCC(C(C)C2CCC3C4=CC(=O)C5CC(OC6OC(CO)C(O)C(...,42.691983,43.961735,49.334545,51.261818,44.888031,49.002041,42.448473,51.036170,48.417607,...,37.772912,36.843022,36.001760,35.452420,39.570915,63.598035,52.922845,23.099889,8.654226,25
1,C/C=C/S(=O)C(CC)SSCCC,72.830189,73.269841,101.574074,107.734694,83.327273,59.983051,49.875000,33.300000,26.400000,...,75.659057,74.887882,78.872284,76.015914,76.624479,48.497812,72.356102,23.556246,10.717787,10
2,COC(=O)C1OC(OC2CCC3(C)C(CCC4(C)C3CC=C3C5CC(C)(...,41.212329,48.534447,50.782904,50.498049,45.957276,51.478378,48.139565,48.682192,49.600852,...,45.842155,70.134041,74.861364,75.661650,71.309912,62.223076,57.948791,22.999512,8.658228,27
3,CCCCC/C=C\C/C=C\CCCCCCCCCC(=O)OC[C@H](COP(=O)(...,57.276018,46.964401,49.495017,55.969799,54.523649,53.853659,58.256140,56.304196,55.315412,...,32.687680,50.078332,52.101558,50.732123,58.659231,68.762972,64.378759,25.040933,8.536598,39
4,CS(=O)CCC(NC(=O)CCC(N)C(=O)O)C(=O)O,94.612903,104.720930,111.988764,109.862500,116.623188,106.609375,109.339286,99.480000,101.833333,...,120.775807,124.122119,131.255377,129.061685,125.552128,50.151560,112.883622,22.472381,9.064450,12


In [15]:
#特征工程
# feature engineer
def FE(df_gapenergy):
    df_gapenergy_fe = df_gapenergy.copy()
    for idx1 in range(10):
        for idx2 in range(idx1+1, 10):
            str_col = str(col_list[idx1])+'*'+str(col_list[idx2])
            df_gapenergy_fe[str_col] = df_gapenergy[col_list[idx1]]*df_gapenergy[col_list[idx2]]
    for idx1 in range(10, 20):
        for idx2 in range(idx1+1, 20):
            str_col = str(col_list[idx1])+'*'+str(col_list[idx2])
            df_gapenergy_fe[str_col] = df_gapenergy[col_list[idx1]]*df_gapenergy[col_list[idx2]]
    for idx1 in range(20,30):
        for idx2 in range(idx1+1, 30):
            str_col = str(col_list[idx1])+'*'+str(col_list[idx2])
            df_gapenergy_fe[str_col] = df_gapenergy[col_list[idx1]]*df_gapenergy[col_list[idx2]]
    for idx1 in range(30,40):
        for idx2 in range(idx1+1, 40):
            str_col = str(col_list[idx1])+'*'+str(col_list[idx2])
            df_gapenergy_fe[str_col] = df_gapenergy[col_list[idx1]]*df_gapenergy[col_list[idx2]]
    
    return df_gapenergy_fe

df_tox_gapenergy_fe = FE(df_tox)
df_exo_gapenergy_fe = FE(df_exo)


/tmp/ipykernel_2838368/2600103942.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_gapenergy_fe[str_col] = df_gapenergy[col_list[idx1]]*df_gapenergy[col_list[idx2]]
/tmp/ipykernel_2838368/2600103942.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_gapenergy_fe[str_col] = df_gapenergy[col_list[idx1]]*df_gapenergy[col_list[idx2]]
/tmp/ipykernel_2838368/2600103942.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [16]:
descList = [i[0] for i in Descriptors._descList]
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descList)

def descri(df):
    descriptors = []
    smis = list(df['standard_smiles'].values)
    for smi in smis:
        mol = Chem.MolFromSmiles(smi)
        descriptors.append(list(calculator.CalcDescriptors(mol)))
    df_des = pd.DataFrame(descriptors, columns=descList )
    df_all = pd.concat([df, df_des], axis=1)
    return df_all

df_tox_all = descri(df_tox_gapenergy_fe)
df_exo_all = descri(df_exo_gapenergy_fe)
df_tox_all.insert(0,'label',np.ones(2560))
df_exo_all.insert(0,'label',np.zeros(7210))

[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
[03:47:39] WARNING: not removing hydrogen atom without neighbors
/tmp/ipykernel_2838368/1470989083.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tox_all.insert(0,'label',np.ones(2560))
/tmp/ipykernel_2838368/1470989083.py:17: Performance

In [17]:
df_tox_all.to_csv('data/tox_gapen_descri_0106_sxnx.csv')
df_exo_all.to_csv('data/exo_gapen_descri_0106_sxnx.csv')